In [1]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

In [5]:
agent = Agent(
    name="Test Agent",
    instructions="A helpful assistant that can perform tasks.",

)
result =await Runner.run(agent, "What is the capital of France?")
print(result.final_output)

The capital of France is Paris.


In [9]:
from agents import Agent,Runner,ModelSettings
from dotenv import load_dotenv

load_dotenv()

# define detailed instructions for weather assistant

weather_instructions="""
You are a weather information assistant who helps user understand weather patterns and phenomena.

YOUR EXPERTISE:
 - Explaining weather concepts and terminology
 - Describing how different weather systems work
 - Answering questions about climate and seasonal patterns
 - Explaining the science behind weather events

LIMITATIONS:
- You cannot provide real time weather forecasts for specific locations
- You don't have the current weather data
- You should not make predictions about future weather events

STYLE:

- Use clear , accessible language that non-meterologists can understand
- Include interesting weather facts when relevant
- Be enthusiastic about meteorology and climate science
"""

# create specialised weather assistant
weather_assistant= Agent(
    name ='WeatherAssist',
    instructions=weather_instructions,
    model="gpt-4.1-mini",
    model_settings= ModelSettings(
        temperature=0.5,
        max_tokens=256
    )
)

In [10]:
result= await Runner.run(
    weather_assistant,"can you tell me about the relationship between climate change and extreme weather events"
)

print(result.final_output)

Absolutely! The relationship between climate change and extreme weather events is a fascinating and important topic in meteorology.

Climate change refers to long-term shifts in temperature, precipitation, and other atmospheric conditions on Earth, primarily driven by increased greenhouse gases like carbon dioxide from human activities. These changes affect the patterns and intensity of weather events.

Here’s how climate change influences extreme weather:

1. **Warmer Temperatures Increase Heatwaves:** As the planet warms, heatwaves become more frequent, intense, and longer-lasting. This happens because higher average temperatures push the extremes upward.

2. **More Intense Storms:** Warmer ocean waters fuel tropical storms and hurricanes, providing more energy that can lead to stronger winds and heavier rainfall. This means storms can become more destructive.

3. **Changes in Rainfall Patterns:** Some regions experience heavier rainfall and flooding, while others may face droughts. 

In [12]:
from agents import Agent, Runner,WebSearchTool
from dotenv import load_dotenv

load_dotenv()

# create a research assistant with web search capabilities
research_assistant= Agent(
    name = "Research Assistant",
    instructions = """
    You're a research assistant who helps users find accurate and up-to-date information on various topics.
    1. Search the web for relevant information using the web search tool.
    2. Synthesize the information you find into a coherent response.
    3. Cite your sources when providing information.
    4. If you cannot find the information, be honest about it.
    5. Structure your responses clearly and logically and in bullet points when appropriate.
    6. Use formal and academic language.
    """,
    tools=[WebSearchTool()],
    model="gpt-4.1-mini",
)

async def research_topic(topic):
    result = await Runner.run(
        research_assistant, f"Please research and summarize the latest findings on the topic: {topic}"
    )
    return result.final_output

summary=await research_topic("Advancements in AI Agens technologies")
print(summary)


Recent advancements in AI agent technologies have led to significant developments across various sectors, enhancing automation, efficiency, and user experience. Key progressions include:

**1. Integration of AI Agents into Business Platforms**

- **Microsoft 365 Copilot**: Microsoft has fully integrated AI agents into Microsoft Teams, SharePoint, and Viva Engage. These agents function proactively, setting meeting agendas, organizing files, guiding discussions, and monitoring task progress, thereby enhancing teamwork and productivity. ([windowscentral.com](https://www.windowscentral.com/artificial-intelligence/microsoft-copilot/microsoft-365-copilot-ai-agents-reach-new-milestone?utm_source=openai))

- **Salesforce's Agentforce**: Salesforce introduced Agentforce, a suite of generative AI agents designed for sales, marketing, commerce, and customer service roles. Powered by the Atlas reasoning engine, these agents aim to operate autonomously within defined parameters, reducing the need f

In [1]:
import os
import requests
from datetime import datetime
from typing import Optional, List
from dataclasses import dataclass

from agents import Agent, Runner, function_tool
from dotenv import load_dotenv

load_dotenv()


@dataclass
class WeatherInfo:
   temperature: float
   feels_like: float
   humidity: int
   description: str
   wind_speed: float
   pressure: int
   location_name: str
   rain_1h: Optional[float] = None
   visibility: Optional[int] = None


@function_tool
def get_weather(lat: float, lon: float) -> str:
   """Get the current weather for a specified location using OpenWeatherMap API.

   Args:
       lat: Latitude of the location (-90 to 90)
       lon: Longitude of the location (-180 to 180)
   """
   # Get API key from environment variables
   WEATHER_API_KEY = os.getenv("OPENWEATHERMAP_API_KEY")

   # Build URL with parameters
   url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={WEATHER_API_KEY}&units=metric"

   try:
       response = requests.get(url)
       response.raise_for_status()
       data = response.json()

       # Extract weather data from the response
       weather_info = WeatherInfo(
           temperature=data["main"]["temp"],
           feels_like=data["main"]["feels_like"],
           humidity=data["main"]["humidity"],
           description=data["weather"][0]["description"],
           wind_speed=data["wind"]["speed"],
           pressure=data["main"]["pressure"],
           location_name=data["name"],
           visibility=data.get("visibility"),
           rain_1h=data.get("rain", {}).get("1h"),
       )

       # Build the response string
       weather_report = f"""
       Weather in {weather_info.location_name}:
       - Temperature: {weather_info.temperature}°C (feels like {weather_info.feels_like}°C)
       - Conditions: {weather_info.description}
       - Humidity: {weather_info.humidity}%
       - Wind speed: {weather_info.wind_speed} m/s
       - Pressure: {weather_info.pressure} hPa
       """
       return weather_report

   except requests.exceptions.RequestException as e:
       return f"Error fetching weather data: {str(e)}"

In [2]:
# Create a weather assistant
weather_assistant = Agent(
   name="Weather Assistant",
   instructions="""You are a weather assistant that can provide current weather information.
  
   When asked about weather, use the get_weather tool to fetch accurate data.
   If the user doesn't specify a country code and there might be ambiguity,
   ask for clarification (e.g., Paris, France vs. Paris, Texas).
  
   Provide friendly commentary along with the weather data, such as clothing suggestions
   or activity recommendations based on the conditions.
   """,
   tools=[get_weather]
)

In [5]:
async def main():
    runner=Runner()
    simple_request=await runner.run(weather_assistant, "What's your capabilties?")
    request_with_location= await runner.run(weather_assistant, "What's the weather in Noida?")
    print(simple_request.final_output)
    print("---"*70)
    print(request_with_location.final_output)

await main()

I’m your weather assistant! Here’s what I can do for you:

- **Current Weather Info:** I can provide up-to-date weather conditions (temperature, wind, humidity, etc.) for any location in the world.
- **Clothing & Activity Tips:** I’ll offer friendly advice on what to wear and fun activities you can enjoy based on the current weather.
- **Travel Weather Checks:** Planning a trip? I can help you check the weather at multiple destinations.
- **Global Assistance:** I handle ambiguous city names by making sure I get the correct country or region (e.g., Paris, France vs. Paris, Texas).

If you’d like to know the weather somewhere, just tell me the city (and country or state if needed), and I’ll get you the current report—with some helpful tips! How can I help you today?
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
The current we

In [6]:
from agents import Agent, Runner
from dotenv import load_dotenv

load_dotenv()

# Specialist agents
note_taking_agent = Agent(
   name="Note Manager",
   instructions="You help users take and organize notes efficiently.",
   # In a real application, this agent would have note-taking tools
)

task_management_agent = Agent(
   name="Task Manager",
   instructions="You help users manage tasks, deadlines, and priorities.",
   # In a real application, this agent would have task management tools
)

aws_agent = Agent(
   name="Task Manager",
   instructions="You help users learn AWS concepts in simple and understandable terms.",
   # In a real application, this agent would have AWS learning tools
)


# Coordinator agent that uses specialists as tools
productivity_assistant = Agent(
   name="Productivity Assistant",
   instructions="""You are a productivity assistant that helps users organize their work and personal life.
  
   For note-taking questions or requests, use the note_taking tool.
   For task and deadline management, use the task_management tool.
   For AWS learning queries, use the aws_agent tool.
  
   Help the user decide which tool is appropriate based on their request,
   and coordinate between different aspects of productivity.
   """,
   tools=[
       note_taking_agent.as_tool(
           tool_name="note_taking",
           tool_description="For taking, organizing, and retrieving notes and information"
       ),
       task_management_agent.as_tool(
           tool_name="task_management",
           tool_description="For managing tasks, setting deadlines, and tracking priorities"
       )
        ,aws_agent.as_tool(
              tool_name="aws_learning",
              tool_description="For learning AWS concepts and services"
         )
   ]
)

In [9]:
async def main():
    runner=Runner()
    result = await runner.run(productivity_assistant, "I need to keep track of my project deadliness and learning AWS EC2.")
    print(result.final_output)

await main()

Here’s how I can help you:

1. **Tracking Project Deadlines:**  
I’ll help you organize and keep track of your project deadlines. Please provide the names of your projects and their corresponding deadlines so I can set them up for you.

2. **Learning AWS EC2:**  
Here’s a quick introduction to AWS EC2:
- EC2 (Elastic Compute Cloud) is Amazon’s service for renting virtual servers in the cloud.
- You pick a server type, launch it, and connect remotely to run your applications or host websites.
- Key concepts include Instances (the servers), AMIs (templates), Key Pairs (for secure access), and Security Groups (firewall rules).
- There’s a free tier to get started for beginners.

Would you like to dive deeper into any specific EC2 topic (like launching your first instance, security, or costs) or get an interactive step-by-step guide?

Let me know:
- Your project details and deadlines for tracking,
- Any specific questions or focus areas for learning AWS EC2!


In [11]:
from pydantic import BaseModel
from typing import List, Optional
from agents import Agent, Runner
from dotenv import load_dotenv

load_dotenv()


# Define person data model
class Person(BaseModel):
   name: str
   role: Optional[str]
   contact: Optional[str]


# Define meeting data model
class Meeting(BaseModel):
   date: str
   time: str
   location: Optional[str]
   duration: Optional[str]


# Define task data model
class Task(BaseModel):
   description: str
   assignee: Optional[str]
   deadline: Optional[str]
   priority: Optional[str]


# Define the complete email data model
class EmailData(BaseModel):
   subject: str
   sender: Person
   recipients: List[Person]
   main_points: List[str]
   meetings: List[Meeting]
   tasks: List[Task]
   next_steps: Optional[str]

In [24]:
# Create an email extraction agent with structured output
email_extractor = Agent(
   name="Email Extractor",
   instructions="""You are an assistant that extracts structured information from emails.
  
   When given an email, carefully identify:
   - Subject and main points
   - People mentioned (names, roles, contact info)
   - Meetings (dates, times, locations)
   - Tasks or action items (with assignees and deadlines)
   - Next steps or follow-ups
  
   Extract this information as structured data. If something is unclear or not mentioned,
   leave those fields empty rather than making assumptions.
   Don't mention the year in the response  and just include date and month.
   """,
   output_type=EmailData,  # This tells the agent to return data in EmailData format
)

In [25]:
sample_email = """
From: Alex Johnson <alex.j@techcorp.com>
To: Team Development <team-dev@techcorp.com>
CC: Sarah Wong <sarah.w@techcorp.com>, Miguel Fernandez <miguel.f@techcorp.com>
Subject: Project Phoenix Update and Next Steps

Hi team,

I wanted to follow up on yesterday's discussion about Project Phoenix and outline our next steps.

Key points from our discussion:
- The beta testing phase has shown promising results with 85% positive feedback
- We're still facing some performance issues on mobile devices
- The client has requested additional features for the dashboard

Let's schedule a follow-up meeting this Friday, Oct 15th at 2:00 PM in Conference Room B. The meeting should last about 1.5 hours, and we'll need to prepare the updated project timeline.

Action items:
1. Sarah to address the mobile performance issues by Oct 20th (High priority)
2. Miguel to create mock-ups for the new dashboard features by next Monday
3. Everyone to review the beta testing feedback document and add comments by EOD tomorrow

If you have any questions before Friday's meeting, feel free to reach out.

Best regards,
Alex Johnson
Senior Project Manager
(555) 123-4567
"""

In [26]:
async def process_email(email_text):
   runner = Runner()
   result = await runner.run(
       email_extractor,
       f"Please extract information from this email:\n\n{email_text}"
   )

   # The result is already a structured EmailData object
   return result


# Process the sample email
result = await process_email(sample_email)

# Display the extracted information
result = result.final_output

print(f"Subject: {result.subject}")
print(f"From: {result.sender.name} ({result.sender.role})")
print("\nMain points:")
for point in result.main_points:
   print(f"- {point}")

print("\nMeetings:")
for meeting in result.meetings:
   print(f"- {meeting.date} at {meeting.time}, Location: {meeting.location}")

print("\nTasks:")
for task in result.tasks:
   print(f"- {task.description}")
   print(
       f"  Assignee: {task.assignee}, Deadline: {task.deadline}, Priority: {task.priority}"
   )


Subject: Project Phoenix Update and Next Steps
From: Alex Johnson (Senior Project Manager)

Main points:
- Beta testing phase has shown 85% positive feedback
- Performance issues on mobile devices persist
- Client requested additional dashboard features

Meetings:
- Oct 15th at 2:00 PM, Location: Conference Room B

Tasks:
- Address the mobile performance issues
  Assignee: Sarah, Deadline: Oct 20th, Priority: High
- Create mock-ups for the new dashboard features
  Assignee: Miguel, Deadline: next Monday, Priority: None
- Review the beta testing feedback document and add comments
  Assignee: Everyone, Deadline: EOD tomorrow, Priority: None
